In [1]:
import  os, sys, time
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('ROOT_FOLDER'))
import config
from oracle_helper import db_oracle

In [7]:
df = pd.read_excel(r"C:\Users\ADMIN\Documents\Map.xlsx", sheet_name= 'MAP')

In [3]:
from config import config
import mssql_helper as ms

In [4]:
engine = config.create_engine_mssql(fast_executemany=True)

In [25]:

update_data=df
table_name= 'test'
mode='upsert'
pk_array= ['PROVINCE']
update_array = []

In [26]:
db_schema = 'data_mart'

In [27]:
cols = update_data.columns

In [7]:
def mapping_type(dtype):
    if dtype.lower() == 'int64':
        return ' INT'
    elif dtype.lower() == 'float64':
        return ' FLOAT'
    elif dtype.lower() == 'object':
        return ' NVARCHAR(100)'
    else:
        return ' NVARCHAR(100)'

In [8]:
types_statement = ''
for i, each_col in enumerate(cols):
    # print(each_col)
    types_statement += str(each_col).lower() + mapping_type(str(update_data[each_col].dtype)) + ','

unique_statement = ''
if len(pk_array) != 0:
    if '.' in table_name:
        unique_statement += f",CONSTRAINT {table_name.split('.')[1].lower()}_unique UNIQUE ({','.join(pk_array)})"
    else:
        unique_statement += f",CONSTRAINT {table_name}_unique UNIQUE ({','.join(pk_array)})"

NameError: name 'cols' is not defined

In [12]:
def mssql_check_table_exits(connection, table_name, db_schema, db_name = config.MSSQL_DWH):
    try:
        connection.execute(f"SELECT top 10 * FROM {db_name}.{db_schema}.{table_name}")
        return True
    except:
        return False

In [16]:
if mssql_check_table_exits(engine, table_name = table_name, db_schema = db_schema) == False:
    
    engine.execute(f"""CREATE TABLE {db_schema}.{table_name} ({types_statement[:-1]} {unique_statement})""")

else:

    engine.execute(f"""DROP TABLE IF EXISTS {db_schema}.{table_name}""")

insert_statement = f"INSERT INTO {db_schema}.{table_name} VALUES ({','.join(['?' for i in cols])})"
engine.execute(insert_statement, update_data.values.tolist())

In [ ]:
engine.execute(f"""DELETE FROM  IF EXISTS {db_schema}.{table_name}""")

In [17]:
f"""DELETE FROM  IF EXISTS {db_schema}.{table_name}"""

'DELETE FROM  IF EXISTS data_mart.test'

In [8]:
df

,PROVINCE,LATITUDE,LONGITUDE,Sale
0,Hà Nội,2.102450e+01,1.058412e+02,936568
1,Bắc Cạn,2.213330e+01,1.058333e+02,851907
2,Bắc Giang,2.127310e+01,1.061947e+02,344443
3,Bắc Ninh,2.118610e+01,1.060763e+02,917216
4,Cao Bẳng,2.267310e+01,1.062500e+02,892913
...,...,...,...,...
61,Long An,1.053220e+01,1.064042e+02,564816
62,Đồng Nai,1.095750e+01,1.068426e+02,368267
63,Hàm Tân,1.234568e+08,1.234568e+08,123456789
64,Tân Xuân,1.234568e+08,1.234568e+08,123456789


In [9]:
df.columns

Index(['PROVINCE', 'LATITUDE', 'LONGITUDE', 'Sale'], dtype='object')

In [10]:
ms.mssql_upsert(engine=engine, update_data=df, table_name= 'test', mode='upsert', pk_array= ['PROVINCE'], update_array=['LONGITUDE', 'Sale'])

In [7]:
sql_merge = """
            MERGE data_mart.test AS Target
            USING staging.test AS Source
                ON Source.PROVINCE = Target.PROVINCE 
            /* new records ('right match') */
            WHEN NOT MATCHED BY Target THEN
                INSERT ( PROVINCE, LATITUDE, LONGITUDE, Sale ) 
                VALUES ( source.PROVINCE , source.LATITUDE , source.LONGITUDE , source.Sale  )
            /* matching records ('inner match') */
            WHEN MATCHED THEN 
                UPDATE SET Target.LATITUDE = Source.LATITUDE , Target.LONGITUDE = Source.LONGITUDE , Target.Sale = Source.Sale 
            /* deprecated records ('left match') */
            WHEN NOT MATCHED BY Source THEN
                DELETE
            ;

 """

In [15]:
conn = engine.connect()

In [16]:
conn.execute(sql_merge)

In [13]:
from psycopg2 import extras, sql

In [14]:
sql.SQL(sql_merge)

SQL("\n            MERGE data_mart.test AS Target\n            USING staging.test AS Source\n                ON Source.PROVINCE = Target.PROVINCE \n            /* new records ('right match') */\n            WHEN NOT MATCHED BY Target THEN\n                INSERT ( PROVINCE, LATITUDE, LONGITUDE, Sale ) \n                VALUES ( source.PROVINCE , source.LATITUDE , source.LONGITUDE , source.Sale  )\n            /* matching records ('inner match') */\n            WHEN MATCHED THEN \n                UPDATE SET Target.LATITUDE = Source.LATITUDE , Target.LONGITUDE = Source.LONGITUDE , Target.Sale = Source.Sale \n            /* deprecated records ('left match') */\n            WHEN NOT MATCHED BY Source THEN\n                DELETE\n            ;\n\n ")

In [10]:
from sqlalchemy.sql import text

In [11]:
text(sql_merge)

In [12]:
engine.execute(text(sql_merge).execution_options(autocommit=True))

In [ ]:
text(sql_merge)

In [7]:
pk_array= ['PROVINCE'] 
update_array = []
cols = df.columns
db_schema = 'data_mart'
table_name = 'test'

'INSERT INTO data_mart.test VALUES (?,?,?,?)'

In [9]:
update_data  = df

In [10]:
update_data.values.tolist()

[['Hà Nội', 21.0245, 105.8412, 936568],
 ['Bắc Cạn', 22.1333, 105.8333, 851907],
 ['Bắc Giang', 21.2731, 106.1947, 344443],
 ['Bắc Ninh', 21.1861, 106.0763, 917216],
 ['Cao Bẳng', 22.6731, 106.25, 892913],
 ['Hà Giang', 22.8233, 104.9836, 1105258],
 ['Hưng Yên', 20.6464, 106.0511, 50242],
 ['Lai Châu', 22.3991, 103.4393, 405441],
 ['Lạng Sơn', 21.8478, 106.7578, 750352],
 ['Lào Cai', 22.4194, 103.995, 1102342],
 ['Thái Nguyên', 21.6, 105.85, 234286],
 ['Vĩnh Phúc', 21.31, 105.5967, 182454],
 ['Hà Nam', 20.5453, 105.9122, 783169],
 ['Hải Dương', 20.9411, 106.3331, 37735],
 ['Hải Phòng', 20.8, 106.6667, 759858],
 ['Quảng Ninh', 20.95, 107.0833, 1075807],
 ['Thái Bình', 20.4461, 106.3422, 1008457],
 ['Nam Định', 20.42, 106.1683, 828898],
 ['Ninh Bình', 20.2539, 105.975, 77804],
 ['Nghệ An', 18.6733, 105.6922, 416263],
 ['Thanh Hóa', 19.8075, 105.7764, 828239],
 ['Hà Tĩnh', 18.3428, 105.9058, 30259],
 ['Điện Biên', 21.3833, 103.0169, 205471],
 ['Hòa Bình', 20.8172, 105.3375, 217543],
 ['Ph

In [14]:
insert_statement = f"INSERT INTO [staging].{table_name} VALUES ({','.join(['?' for i in cols])})"


In [15]:
engine.execute(insert_statement, update_data.values.tolist())  

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'staging.test'. (208) (SQLExecute); [42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Statement(s) could not be prepared. (8180)")
[SQL: INSERT INTO [staging].test VALUES (?,?,?,?)]
[parameters: [('Hà Nội', 21.0245, 105.8412, 936568), ('Bắc Cạn', 22.1333, 105.8333, 851907), ('Bắc Giang', 21.2731, 106.1947, 344443), ('Bắc Ninh', 21.1861, 106.0763, 917216), ('Cao Bẳng', 22.6731, 106.25, 892913), ('Hà Giang', 22.8233, 104.9836, 1105258), ('Hưng Yên', 20.6464, 106.0511, 50242), ('Lai Châu', 22.3991, 103.4393, 405441)  ... displaying 10 of 63 total bound parameter sets ...  ('Long An', 10.5322, 106.4042, 564816), ('Đồng Nai', 10.9575, 106.8426, 368267)]]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [11]:

engine.execute(insert_statement, update_data.values.tolist())

IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'test_unique'. Cannot insert duplicate key in object 'data_mart.test'. The duplicate key value is (Hà Nội). (2627) (SQLExecute); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'test_unique'. Cannot insert duplicate key in object 'data_mart.test'. The duplicate key value is (Bắc Cạn). (2627); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'test_unique'. Cannot insert duplicate key in object 'data_mart.test'. The duplicate key value is (Bắc Giang). (2627); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has bee

In [15]:
mapping_statement = 'and '.join(f'Source.{i} = Target.{i} ' for i in pk_array)
update_cols = update_array if len(update_array) > 0 else [x for x in list(cols) + pk_array if x in list(cols) and x not in pk_array]
update_sratement = 'and '.join(f'Target.{i} = Source.{i} ' for i in update_cols)
sql_merge = f"""
    MERGE {db_schema}.{table_name} AS Target
    USING staging.{table_name} AS Source
        ON {mapping_statement}
    /* new records ('right match') */
    WHEN NOT MATCHED BY Target THEN
        INSERT ( { ', '.join(cols) } ) 
        VALUES ( { ', '.join(f'source.{i} ' for i in cols)} )
    /* matching records ('inner match') */
    WHEN MATCHED THEN 
        UPDATE SET {update_sratement}
    /* deprecated records ('left match') */
    WHEN NOT MATCHED BY Source THEN
        DELETE
"""

In [16]:
print(sql_merge)


    MERGE data_mart.test AS Target
    USING staging.test AS Source
        ON Source.PROVINCE = Target.PROVINCE 
    /* new records ('right match') */
    WHEN NOT MATCHED BY Target THEN
        INSERT ( PROVINCE, LATITUDE, LONGITUDE, Sale ) 
        VALUES ( source.PROVINCE , source.LATITUDE , source.LONGITUDE , source.Sale  )
    /* matching records ('inner match') */
    WHEN MATCHED THEN 
        UPDATE SET Target.LATITUDE = Source.LATITUDE and Target.LONGITUDE = Source.LONGITUDE and Target.Sale = Source.Sale 
    /* deprecated records ('left match') */
    WHEN NOT MATCHED BY Source THEN
        DELETE

